In [1]:
import os
from tifffile import imread, imwrite
import numpy as np
import pandas as pd
from skimage.draw import polygon

In [2]:
input_dir = r'R:\CellDive\BLCA-1B\BLCA-1B_Final'
df_path = r'R:\Wayne\BLCA\BLCA-1B_Analysis\cores.pkl'
#output_dir = r'R:\Wayne\BLCA\BLCA-5_Analysis\Core Storage'
#selection_list = None

In [3]:
df = pd.read_pickle(df_path)
df.head()

,core_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,Core_000,640.0,4992.0,3136.0,8832.0,polygon,"[[2688.0, 6080.0], [4992.0, 3136.0], [640.0, 3..."
1,Core_001,960.0,5632.0,9920.0,15680.0,rectangle,"[[5632.0, 15680.0], [5632.0, 9920.0], [960.0, ..."
2,Core_002,1664.0,5504.0,16896.0,22656.0,rectangle,"[[5504.0, 22656.0], [5504.0, 16896.0], [1664.0..."
3,Core_003,1728.0,5952.0,23680.0,29312.0,rectangle,"[[5952.0, 29312.0], [5952.0, 23680.0], [1728.0..."
4,Core_004,1856.0,6400.0,30400.0,35712.0,rectangle,"[[6400.0, 35712.0], [6400.0, 30400.0], [1856.0..."


In [4]:
import zarr
import dask.array as da


# pathway to the image for segmentation
im_path = r'R:\CellDive\BLCA-1B\BLCA-1B_Final\BLCA-1B_1.0.4_R000_DAPI__FINAL_F.ome.tif'

store = imread(im_path,aszarr=True)
group = zarr.open(store, mode='r')
zattrs = group.attrs.asdict()
level_num = 0

path = zattrs['multiscales'][0]['datasets'][level_num]['path']
im = da.from_zarr(group[path])

In [5]:
%load_ext autoreload
%autoreload 2

from multiplex_pipeline.core_cutting.cutter import CoreCutter
cutter = CoreCutter(margin=0, mask_value=0)
test = cutter.extract_core(array=im, row=df.loc[1])

In [6]:
test

array([[114, 107,  97, ...,  98,  98,  98],
       [114, 109, 114, ...,  98,  98,  98],
       [117, 117, 128, ...,  98,  98,  98],
       ...,
       [350, 386, 378, ...,  98,  98,  98],
       [369, 345, 400, ...,  98,  98,  98],
       [385, 443, 383, ...,  98,  98,  98]], dtype=uint16)

In [7]:
from spatialdata import SpatialData
from xarray import DataArray
from spatialdata.models import Image2DModel

c:\Users\KMK280\AppData\Local\miniconda3\envs\napari-env\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [21]:
max_pyramid_levels = 0
scale_factors=[2**x for x in range(1,max_pyramid_levels) if max_pyramid_levels > 1]
scale_factors

[]

In [15]:
parsed_image = Image2DModel.parse(np.expand_dims(test, axis=0), dims=("c","y","x"),scale_factors=[])
Image2DModel().validate(parsed_image)

In [16]:
sd =SpatialData()
my_images = {"im1": parsed_image}
sd = SpatialData(images=my_images)
sd

SpatialData object
└── Images
      └── 'im1': DataTree[cyx] (1, 4672, 5760)
with coordinate systems:
    ▸ 'global', with elements:
        im1 (Images)

In [62]:
sd.images

{'im1': <xarray.DataArray 'image' (c: 2, y: 4672, x: 5760)> Size: 108MB
dask.array<array, shape=(2, 4672, 5760), dtype=uint16, chunksize=(2, 4672, 5760), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) <U2 16B 't1' 't2'
  * y        (y) float64 37kB 0.5 1.5 2.5 3.5 ... 4.67e+03 4.67e+03 4.672e+03
  * x        (x) float64 46kB 0.5 1.5 2.5 3.5 ... 5.758e+03 5.758e+03 5.76e+03
Attributes:
    transform:  {'global': Identity }}

In [8]:
%load_ext autoreload
%autoreload 2

from multiplex_pipeline.core_cutting.controller import CoreController

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
controller = CoreController(
    metadata_df=df.loc[:2,:],
    image_dir=r'R:\CellDive\BLCA-1B\BLCA-1B_Final',
    temp_dir=os.getcwd(),
    output_dir=os.getcwd(),
    max_pyramid_levels=3,
    allowed_channels = ['CD3','002_DAPI'],
)
controller.run()

Found 2 channels to process.
Processing channel: CD3
Processing channel: 002_DAPI
Assembling SpatialData for: Core_000
INFO     The Zarr backing store has been changed from None the new file path:                                      
         d:\multiplex_pipeline\notebooks\Core_000.zarr                                                             
Assembling SpatialData for: Core_001
INFO     The Zarr backing store has been changed from None the new file path:                                      
         d:\multiplex_pipeline\notebooks\Core_001.zarr                                                             
Assembling SpatialData for: Core_002
INFO     The Zarr backing store has been changed from None the new file path:                                      
         d:\multiplex_pipeline\notebooks\Core_002.zarr                                                             


Awesome, now let's deal with the file management. So, the files may be read in and saved locally or through Globus. If Globus is used than spatialdata 

In [20]:
os.getcwd()

'd:\\multiplex_pipeline\\notebooks'

In [22]:
from pathlib import Path

In [29]:
%%time
f = Path("d:\\multiplex_pipeline\\notebooks\Core_000.zarr")
sd.write(f, overwrite=True)


ValueError: Cannot overwrite. The target path of the write operation is in use. Please save the data to a different location. 
Details: the target path contains one or more files that Dask use for backing elements in the SpatialData object.
Workaround: please see discussion here https://github.com/scverse/spatialdata/discussions/520 .

In [35]:
f = Path("d:\\multiplex_pipeline\\notebooks\Core_000.zarr")
sd = SpatialData.read(f)
sd

version mismatch: detected: RasterFormatV02, requested: FormatV04
c:\Users\KMK280\AppData\Local\miniconda3\envs\napari-env\Lib\site-packages\zarr\creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
version mismatch: detected: RasterFormatV02, requested: FormatV04


SpatialData object, with associated Zarr store: D:\multiplex_pipeline\notebooks\Core_000.zarr
└── Images
      ├── '002_DAPI': DataTree[cyx] (1, 4352, 5696), (1, 2176, 2848), (1, 1088, 1424), (1, 544, 712)
      └── 'CD3': DataTree[cyx] (1, 4352, 5696), (1, 2176, 2848), (1, 1088, 1424), (1, 544, 712)
with coordinate systems:
    ▸ 'global', with elements:
        002_DAPI (Images), CD3 (Images)

In [31]:
[2]*0

[]

In [50]:
sd["im3"] = parsed_image
sd

SpatialData object, with associated Zarr store: D:\data_tt.zarr
└── Images
      ├── 'im1': DataArray[cyx] (1, 4672, 5760)
      ├── 'im2': DataArray[cyx] (1, 4672, 5760)
      └── 'im3': DataArray[cyx] (1, 4672, 5760)
with coordinate systems:
    ▸ 'global', with elements:
        im1 (Images), im2 (Images), im3 (Images)
with the following elements not in the Zarr store:
    ▸ im3 (Images)

In [45]:
sd

SpatialData object, with associated Zarr store: D:\data_tt.zarr
└── Images
      └── 'im1': DataArray[cyx] (1, 4672, 5760)
with coordinate systems:
    ▸ 'global', with elements:
        im1 (Images)

In [47]:
%%time
sd.write_element("im2")

CPU times: total: 62.5 ms
Wall time: 191 ms


In [7]:
import napari
viewer = napari.Viewer()
viewer.add_image(test)

<Image layer 'test' at 0x1f77e8a63d0>

In [4]:
# keep only the selected cores
if selection_list:
    df = df[df['core_id'].isin(selection_list[0])]

# get list of subdirectories
dir_list = os.listdir(input_dir)

for ind,core in df.iterrows():
    if core.core_name not in dir_list:
        os.makedirs(os.path.join(output_dir,core.core_name), exist_ok=True)

# get the list of images
im_list = [x for x in os.listdir(input_dir) if ('.ome.tif' in x and '.0.4_R' in x)]

mask_dict = {}

for im_name in im_list:
    print(im_name)
    im = imread(os.path.join(input_dir,im_name))

    for ind,core in df.iterrows():

        core_im = im[core.row_start:core.row_stop,core.column_start:core.column_stop]

        if core.poly_type != 'rectangle':

            mask = mask_dict.get(core.core_name)

            if mask is None:
            
                mask = np.zeros_like(core_im, dtype=bool)
                r, c = polygon(core.polygon_vertices[:, 1], core.polygon_vertices[:, 0], mask.shape)

                mask[r, c] = True

                mask_dict[core.core_name] = mask

            core_im[~mask] = 0

        # save the core
        core_name = im_name.replace('.ome.tif',f'_{core.core_name}.ome.tif')
        imwrite(os.path.join(output_dir,core.core_name,core_name),core_im)

BLCA-5_1.0.4_R000_Cy3_pH2AX-AF555_FINAL_AFR_F.ome.tif
BLCA-5_1.0.4_R000_Cy5_pRB-AF647_FINAL_AFR_F.ome.tif
BLCA-5_1.0.4_R000_Cy7_cycD3-AF750_FINAL_AFR_F.ome.tif
BLCA-5_1.0.4_R000_DAPI__FINAL_F.ome.tif
BLCA-5_1.0.4_R000_FITC_CDC25C-AF488_FINAL_AFR_F.ome.tif
BLCA-5_10.0.4_R000_Cy3_p38-AF555_FINAL_AFR_F.ome.tif
BLCA-5_10.0.4_R000_Cy5_p16-AF647_FINAL_AFR_F.ome.tif
BLCA-5_10.0.4_R000_Cy7_PDL1-AF750_FINAL_AFR_F.ome.tif
BLCA-5_10.0.4_R000_DAPI__FINAL_F.ome.tif
BLCA-5_10.0.4_R000_FITC_CD127-AF488_FINAL_AFR_F.ome.tif
BLCA-5_11.0.4_R000_Cy3_CD3-AF555_FINAL_AFR_F.ome.tif
BLCA-5_11.0.4_R000_Cy5_SKP2-AF647_FINAL_AFR_F.ome.tif
BLCA-5_11.0.4_R000_Cy7_CD45-AF750_FINAL_AFR_F.ome.tif
BLCA-5_11.0.4_R000_DAPI__FINAL_F.ome.tif
BLCA-5_11.0.4_R000_FITC_CCR7-AF488_FINAL_AFR_F.ome.tif
BLCA-5_12.0.4_R000_Cy3_CD11C-AF555_FINAL_AFR_F.ome.tif
BLCA-5_12.0.4_R000_Cy5_cMYC-AF647_FINAL_AFR_F.ome.tif
BLCA-5_12.0.4_R000_Cy7_CD20-AF750_FINAL_AFR_F.ome.tif
BLCA-5_12.0.4_R000_DAPI__FINAL_F.ome.tif
BLCA-5_12.0.4_R000_FITC_IL

In [5]:
479/60


7.983333333333333